## Deliverable 2. Create a Customer Travel Destinations Map.

In [81]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [82]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Burnie,AU,-41.0667,145.9167,61.95,85,9,6.20,clear sky
1,1,Jamestown,US,42.0970,-79.2353,24.12,51,100,8.05,mist
2,2,Bakel,SN,14.9050,-12.4568,95.38,7,82,2.62,broken clouds
3,3,Cape Town,ZA,-33.9258,18.4232,78.12,46,0,29.93,clear sky
4,4,Kavieng,PG,-2.5744,150.7967,82.51,76,36,9.15,light rain


In [83]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [84]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Cape Town,ZA,-33.9258,18.4232,78.12,46,0,29.93,clear sky
4,4,Kavieng,PG,-2.5744,150.7967,82.51,76,36,9.15,light rain
7,7,Arraial Do Cabo,BR,-22.9661,-42.0278,85.96,74,75,8.05,broken clouds
14,14,Atuona,PF,-9.8000,-139.0333,77.16,76,3,14.88,clear sky
20,20,Rikitea,PF,-23.1203,-134.9692,77.38,77,100,17.09,light rain
22,22,Hermanus,ZA,-34.4187,19.2345,77.90,61,3,13.58,clear sky
26,26,Geraldton,AU,-28.7667,114.6000,75.61,78,16,14.97,few clouds
35,35,Vaini,TO,-21.2000,-175.2000,82.18,80,95,20.31,overcast clouds
49,49,Faanui,PF,-16.4833,-151.7500,81.07,74,8,16.40,light rain
50,50,Touros,BR,-5.1989,-35.4608,83.30,61,68,8.08,broken clouds


In [85]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             211
City                   211
Country                210
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [86]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.count()

Unnamed: 0             210
City                   210
Country                210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [87]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Cape Town,ZA,78.12,clear sky,-33.9258,18.4232,
4,Kavieng,PG,82.51,light rain,-2.5744,150.7967,
7,Arraial Do Cabo,BR,85.96,broken clouds,-22.9661,-42.0278,
14,Atuona,PF,77.16,clear sky,-9.8000,-139.0333,
20,Rikitea,PF,77.38,light rain,-23.1203,-134.9692,
22,Hermanus,ZA,77.90,clear sky,-34.4187,19.2345,
26,Geraldton,AU,75.61,few clouds,-28.7667,114.6000,
35,Vaini,TO,82.18,overcast clouds,-21.2000,-175.2000,
49,Faanui,PF,81.07,light rain,-16.4833,-151.7500,
50,Touros,BR,83.30,broken clouds,-5.1989,-35.4608,


In [88]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Cape Town,ZA,78.12,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
4,Kavieng,PG,82.51,light rain,-2.5744,150.7967,Nusa Island Retreat
7,Arraial Do Cabo,BR,85.96,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
14,Atuona,PF,77.16,clear sky,-9.8000,-139.0333,Villa Enata
20,Rikitea,PF,77.38,light rain,-23.1203,-134.9692,People ThankYou
22,Hermanus,ZA,77.90,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
26,Geraldton,AU,75.61,few clouds,-28.7667,114.6000,Broadwater Mariner Resort
35,Vaini,TO,82.18,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
49,Faanui,PF,81.07,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
50,Touros,BR,83.30,broken clouds,-5.1989,-35.4608,INN NEW HORIZON


In [93]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [94]:
# 8a. Create the output File (CSV)
# 8b. Export the City_Data into a csv
output_data_file = clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [95]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [96]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))